In [18]:
import requests
from pathlib import Path
import pandas as pd



In [26]:

current_dir = Path.cwd()
api_key_path = current_dir / "cmc.txt"

with open(api_key_path, 'r') as f:
    api_key = f.read().strip()

def fetch_global_market_cap() -> dict:
    url = 'https://pro-api.coinmarketcap.com/v1/global-metrics/quotes/latest'
    headers = {
        'X-CMC_PRO_API_KEY': api_key,
        'Accept': 'application/json'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

In [27]:
data = fetch_global_market_cap()

In [31]:
dataframe

,status,data
timestamp,2025-09-02T14:21:11.162Z,NaN
error_code,0,NaN
error_message,None,NaN
elapsed,17,NaN
credit_count,1,NaN
notice,None,NaN
active_cryptocurrencies,NaN,9512
total_cryptocurrencies,NaN,35924
active_market_pairs,NaN,108677
active_exchanges,NaN,846
